In [15]:
import pandas as pd
import json
import os

In [16]:
# Definir la ruta base de los archivos
# Asegúrate de cambiar esta ruta si el dataset está en otra ubicación
RUTA_BASE = "C:/Users/oscar/Desktop/Henry/Proyectos/INDIVIDUAL 1/data_set"

In [17]:

#  Cargar el archivo credits.csv
ruta_movies = os.path.join(RUTA_BASE, "movies_dataset.csv")
movies_df = pd.read_csv(ruta_movies, low_memory=False)

In [18]:
#  Ver las primeras filas
display(movies_df.head())

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [19]:
#  Eliminar registros con 'release_date' mal formateado
# Se eliminan valores nulos y se filtran fechas con formato incorrecto
movies_df = movies_df.dropna(subset=['release_date'])
movies_df = movies_df[movies_df['release_date'].str.match(r'^\d{4}-\d{2}-\d{2}$', na=False)]

In [20]:
# Convertir la columna 'release_date' a formato datetime para mejor manejo de fechas
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], format='%Y-%m-%d')
movies_df['release_year'] = movies_df['release_date'].dt.year


In [21]:
#  Manejo de valores nulos en columnas numéricas
# Se convierten a valores numéricos y se rellenan nulos con 0
movies_df['revenue'] = pd.to_numeric(movies_df['revenue'], errors='coerce').fillna(0)
movies_df['budget'] = pd.to_numeric(movies_df['budget'], errors='coerce').fillna(0)


In [22]:
#  Calcular el retorno de inversión (ROI) como revenue / budget
# Si el presupuesto es 0, se asigna un ROI de 0 para evitar divisiones por cero
movies_df['return'] = movies_df.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] > 0 else 0, axis=1)


In [23]:
#  Filtrar solo películas con estado 'Released'
movies_df = movies_df[movies_df['status'] == 'Released'].drop(columns=['status'])


In [24]:
#  Convertir 'id' a entero para facilitar su uso en fusiones con el dataset credits.csv
movies_df['id'] = movies_df['id'].astype(int)


In [25]:
#  Función para desanidar columnas JSON y extraer nombres
def extract_names(json_str):
    if pd.isna(json_str):
        return None
    if not isinstance(json_str, str):
        return None
    try:
        data = json.loads(json_str.replace("'", '"'))
        if isinstance(data, list):
            return ', '.join([item.get('name', '') for item in data])
        elif isinstance(data, dict):
            return data.get('name', '')
    except json.JSONDecodeError:
        return None
    return None


In [26]:
#  Aplicar la función 'extract_names' a las columnas JSON relevantes
json_columns = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']
for col in json_columns:
    if col in movies_df.columns:
        movies_df[col] = movies_df[col].apply(extract_names)

In [27]:
# Eliminar columnas innecesarias para optimización
columnas_a_eliminar = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
movies_df = movies_df.drop(columns=[col for col in columnas_a_eliminar if col in movies_df.columns])


In [28]:
#  Guardar el archivo transformado en formato Parquet para mejor eficiencia
ruta_movies_parquet = os.path.join(RUTA_BASE, "movies_dataset_etl.parquet")
movies_df.to_parquet(ruta_movies_parquet, index=False)

print(f" Transformación de `movies_dataset.csv` completada. Archivo guardado en:\n  {ruta_movies_parquet}")

 Transformación de `movies_dataset.csv` completada. Archivo guardado en:
  C:/Users/oscar/Desktop/Henry/Proyectos/INDIVIDUAL 1/data_set\movies_dataset_etl.parquet
